In [4]:
!nvidia-smi

Wed Dec 25 03:04:03 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P8              12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [5]:
import os
HOME = os.getcwd()
print(HOME)

/content


In [6]:
%pip install ultralytics supervision roboflow
import ultralytics
ultralytics.checks()

Ultralytics 8.3.54 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 32.7/112.6 GB disk)


In [8]:
from ultralytics import YOLO
from PIL import Image
import requests

model = YOLO('yolo11n-seg.pt')
image = Image.open('5.jpg')
result = model.predict(image, conf= 0.25)[0]


0: 320x640 6 cars, 1 bus, 2 trucks, 111.8ms
Speed: 33.0ms preprocess, 111.8ms inference, 1103.6ms postprocess per image at shape (1, 3, 320, 640)


In [15]:
!mkdir {HOME}/datasets
%cd {HOME}/datasets

from google.colab import userdata
from roboflow import Roboflow

ROBOFLOW_API_KEY = userdata.get('ROBOFLOW_API_KEY')
rf = Roboflow(api_key=ROBOFLOW_API_KEY)

project = rf.workspace("aditya-choudhary-ehv9p").project("l-s-kvbur")
version = project.version(1)
dataset = version.download("yolov11", overwrite=True)

mkdir: cannot create directory ‘/content/datasets’: File exists
/content/datasets
loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to L-S-1 in yolov11:: 100%|██████████| 2050/2050 [00:00<00:00, 3216.63it/s]


In [16]:
dataset.location

'/content/datasets/L-S-1'

In [17]:
%cd {HOME}

!yolo task = segment mode = train model = yolo11m-seg.pt data = {dataset.location}/data.yaml epochs = 30 imgsz = 640 plots = True

/content
100% 43.3M/43.3M [00:02<00:00, 20.6MB/s]
Ultralytics 8.3.54 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=segment, mode=train, model=yolo11m-seg.pt, data=/content/datasets/L-S-1/data.yaml, epochs=30, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False,

In [18]:
!yolo segment predict model=yolo11n-seg.pt source='/content/5.jpg'  # predict with official model

Ultralytics 8.3.54 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLO11n-seg summary (fused): 265 layers, 2,868,664 parameters, 0 gradients, 10.4 GFLOPs

image 1/1 /content/5.jpg: 320x640 6 cars, 1 bus, 2 trucks, 59.4ms
Speed: 2.4ms preprocess, 59.4ms inference, 708.1ms postprocess per image at shape (1, 3, 320, 640)
Results saved to runs/segment/predict
💡 Learn more at https://docs.ultralytics.com/modes/predict
